In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import cv2
import os
import tensorflow as tf
import json
import crop

from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

### _04 : 오른쪽 눈 주름(r_perocular_wrinkle)에 대해 bbox_crop

In [4]:
label_key = "_F_03.json"
annotation_key = "l_perocular_wrinkle"
image_data, target_data = crop.process_files(label_key, annotation_key)

Error processing file D:\final_project_backup\FINAL_DATA\한국인 피부상태 측정 데이터\Training\labels\pad\0138\0138_02_F_03.json: 'NoneType' object is not iterable
Error processing file D:\final_project_backup\FINAL_DATA\한국인 피부상태 측정 데이터\Training\labels\pad\0823\0823_02_F_03.json: 'NoneType' object is not iterable


In [5]:
print(f"Number of images processed: {len(image_data)}")
print(f"Number of targets processed: {len(target_data)}")

Number of images processed: 2572
Number of targets processed: 2572


### 데이터 저장 및 불러오기

In [6]:
# 이미지, 타겟 데이터 저장
with open(f'{annotation_key}_image_data.pkl', 'wb') as f:
    pickle.dump(image_data, f)

with open(f'{annotation_key}_target_data.pkl', 'wb') as f:
    pickle.dump(target_data, f)

In [7]:
# 데이터 로드
with open(f'{annotation_key}_image_data.pkl', 'rb') as f:
    image_data = pickle.load(f)

with open(f'{annotation_key}_target_data.pkl', 'rb') as f:
    target_data = pickle.load(f)

In [8]:
from collections import Counter

values_list = list(target_data.values())
value_counts = Counter(values_list)

for value, count in value_counts.items():
    print(f"Value: {value}, Count: {count}")

Value: 3, Count: 380
Value: 0, Count: 180
Value: 1, Count: 863
Value: 4, Count: 231
Value: 2, Count: 384
Value: 5, Count: 270
Value: 6, Count: 264


### 이진분류(0,1)과 (3,4,5,6)로 분류

In [41]:
def merge_classes(target_data):
    binary_target_data = {}
    for key, value in target_data.items():
        if value in [0, 1]:
            binary_target_data[key] = '10'
        elif value in [3, 4, 5, 6]:
            binary_target_data[key] = '20'
        else:
            binary_target_data[key] = value
    
    return binary_target_data

binary_target_data = merge_classes(target_data)

In [42]:
filtered_target_data = {key: value for key, value in binary_target_data.items() if value in ['10', '20']}

values_list = list(filtered_target_data.values())
value_counts = Counter(values_list)

for value, count in value_counts.items():
    print(f"Value: {value}, Count: {count}")

Value: 20, Count: 1145
Value: 10, Count: 1043


In [43]:
common_keys = set(image_data.keys()) & set(filtered_target_data.keys())

# 데이터 정렬
filtered_image_data = {key: image_data[key] for key in common_keys}
filtered_target_data = {key: filtered_target_data[key] for key in common_keys}

# X와 y 생성
X = np.array(list(filtered_image_data.values()))
y = np.array(list(filtered_target_data.values()))

print(f"Length of X: {len(X)}, Length of y: {len(y)}")

Length of X: 2188, Length of y: 2188


In [44]:
class_mapping = {'10': 0, '20': 1}
y = np.array([class_mapping[label] for label in y])
X = X / 255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (1750, 128, 128, 3), y_train shape: (1750,)
X_test shape: (438, 128, 128, 3), y_test shape: (438,)


In [45]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], X.shape[3])),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # 이진 분류 출력층
])

# 모델 컴파일
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(learning_rate=0.0001), 
              metrics=['accuracy'])

# 모델 요약
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 14, 14, 128)      

In [46]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)


In [47]:
# 모델 학습
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    class_weight = class_weights,
    callbacks=[early_stopping]
)

Epoch 1/100
44/44 [==============================] - 16s 344ms/step - loss: 0.6964 - accuracy: 0.5243 - val_loss: 0.6932 - val_accuracy: 0.4886
Epoch 2/100
44/44 [==============================] - 13s 293ms/step - loss: 0.6916 - accuracy: 0.5186 - val_loss: 0.6910 - val_accuracy: 0.5886
Epoch 3/100
44/44 [==============================] - 14s 308ms/step - loss: 0.6910 - accuracy: 0.5307 - val_loss: 0.6888 - val_accuracy: 0.5943
Epoch 4/100
44/44 [==============================] - 13s 285ms/step - loss: 0.6909 - accuracy: 0.5136 - val_loss: 0.6914 - val_accuracy: 0.5200
Epoch 5/100
44/44 [==============================] - 12s 268ms/step - loss: 0.6872 - accuracy: 0.5371 - val_loss: 0.6879 - val_accuracy: 0.5400
Epoch 6/100
44/44 [==============================] - 13s 289ms/step - loss: 0.6842 - accuracy: 0.5357 - val_loss: 0.6838 - val_accuracy: 0.5029
Epoch 7/100
44/44 [==============================] - 11s 248ms/step - loss: 0.6807 - accuracy: 0.5486 - val_loss: 0.6785 - val_accuracy:

In [48]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

14/14 - 1s - loss: 0.3376 - accuracy: 0.8630 - 793ms/epoch - 57ms/step
Test Loss: 0.3376486003398895
Test Accuracy: 0.8630136847496033


In [49]:
# 예측 수행 (확률 값 반환)
predictions = model.predict(X_test)  # 이진 분류에서는 sigmoid 출력

# 0.5를 기준으로 클래스 결정
predicted_classes = (predictions > 0.5).astype(int).flatten()

print("Predicted Classes: ", predicted_classes)


14/14 [==============================] - 1s 60ms/step
Predicted Classes:  [0 0 0 0 0 1 0 0 0 1 1 1 0 1 1 0 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 1 1 0 1 0 1
 1 0 1 0 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 1
 1 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 1 1 0 0 1 1 0 0 1 1 0 1 0 1 0 1
 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0
 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 1 1 0 0 1 0 0 1 0 0 1 0
 1 0 0 0 0 0 0 1 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0
 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 1 0 1 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0
 1 1 1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 0 1 1 1 1
 1 0 0 1 0 1 1 1 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0
 1 1 0 0 1 0 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 0 1 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0
 0 1 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 0 0 0 1 1 0 1 1 0 1 1 1 0 1]
